In [ ]:
!pip install langchain_openai
!pip install langchain_google_genai
!pip install cassandra-driver python-dotenv
!pip install astrapy
!pip install langchain-community
!pip install google-auth google-auth-httplib2 google-auth-oauthlib google-api-python-client langchain langchain_google_genai

In [52]:
import os
import csv
import json
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from astrapy import DataAPIClient
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from google.auth import load_credentials_from_file
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.schema import BaseOutputParser

In [ ]:
# Loading credentials from the JSON file
json_file_path = "/content/social_media_db-token.json"
with open(json_file_path, 'r') as file:
    credentials = json.load(file)

ASTRA_CLIENT_ID = credentials["clientId"]
ASTRA_CLIENT_SECRET = credentials["secret"]
SECURE_CONNECT_BUNDLE = "/content/secure-connect-social-media-db.zip"  
TABLE_NAME = "engagement_table"
KEYSPACE = "engagement_data"

# Connecting to AstraDB
auth_provider = PlainTextAuthProvider(ASTRA_CLIENT_ID, ASTRA_CLIENT_SECRET)
cluster = Cluster(cloud={'secure_connect_bundle': SECURE_CONNECT_BUNDLE}, auth_provider=auth_provider)

try:
    session = cluster.connect()
    session.set_keyspace(KEYSPACE)
    print(f"Connected to keyspace: {KEYSPACE}")
except Exception as e:
    print(f"Error connecting to AstraDB: {e}")
    raise

# Checking if the table exists or not
def check_or_create_table():
    try:
        query = f"SELECT table_name FROM system_schema.tables WHERE keyspace_name='{KEYSPACE}' AND table_name='{TABLE_NAME}'"
        rows = session.execute(query)
        if rows.one():
            print(f"Table '{TABLE_NAME}' already exists.")
        else:
            print(f"Table '{TABLE_NAME}' does not exist.")

    except Exception as e:
        print(f"Error checking or creating table: {e}")
        raise

check_or_create_table()


In [ ]:
import csv
import uuid

# Table schema
TABLE_NAME = "engagement_table"
file_path = "/content/dataset.csv" 

try:
    with open(file_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        c = 1
        for row in csv_reader:
            try:
                post_id = uuid.UUID(row["post_id"]) if row["post_id"] else uuid.uuid4()

                insert_query = f"""
                INSERT INTO {TABLE_NAME} (post_id, comments, likes, post_type, shares)
                VALUES (%s, %s, %s, %s, %s)
                """

                session.execute(
                    insert_query,
                    (
                        post_id,
                        int(row["comments"]),
                        int(row["likes"]),
                        row["post_type"],
                        int(row["shares"]),
                    ),
                )
                print(f"Processed row {c}")
                c += 1
            except KeyError as e:
                print(f"Error: Missing required column in the file: {e}")
            except ValueError as e:
                print(f"Error: Invalid data format for row {c}: {e}")
            except Exception as e:
                print(f"An error occurred while processing row {c}: {e}")

        print("Data successfully inserted into the database.")
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")


In [55]:

def fetch_data_from_db():
    TABLE_NAME = "engagement_table"
    try:
        select_query = f"SELECT * FROM {TABLE_NAME}"
        rows = session.execute(select_query)
        data = [dict(row._asdict()) for row in rows]
        return data
    except Exception as e:
        print(f"Error retrieving data from Astra DB: {e}")
        return []


In [ ]:


# Load credentials from our service account key file
def load_google_credentials():
    # Loading the credentials from our JSON file
    credentials, project = load_credentials_from_file('/content/chat-bot-flexi-555a57f8ae6e.json')

    credentials = credentials.with_scopes([
        "https://www.googleapis.com/auth/cloud-platform",
        "https://www.googleapis.com/auth/generative-language"
    ])
    return credentials

# Preparing the model with Google API credentials
def get_gemini_model():
    credentials = load_google_credentials()
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", credentials=credentials)
    return llm


def generate_prompt():
    prompt_template = (
        "Here is the entire dataset: {data}. "
        "Identify patterns, trends, and insights from this dataset. "
        "Summarize the key trends only without providing row-by-row details. "
        "For example, one trend could be 'Posts by females receive 46% more likes compared to posts by males'. "
        "Similarly, generate more insights from the data and use numbers to further strengthen the trends. "
        "Finally, the output should be a list of trends and insights from the data. Do not generate anything else, no suggestions. "
        "Generate as many trends as possible, including all columns of the data, except the _id and post_id."
    )
    prompt = PromptTemplate(input_variables=["data"], template=prompt_template)
    return prompt

# Setting up the chain with LLM and prompt
def setup_chain():
    llm = get_gemini_model()
    prompt = generate_prompt()
    chain = (
        {"data": RunnablePassthrough()} 
        | prompt
        | llm  
        | StrOutputParser()  
    )
    return chain


formatted_data = fetch_data_from_db()

# Run the chain to generate insights
def generate_insights():
    chain = setup_chain()
    output = chain.invoke(input=formatted_data)  
    print("Generated Insights:")
    print(output)

if __name__ == "__main__":
    generate_insights()
